In [1]:
import pandas as pd
import geopandas as gpd
import folium
import branca
from matplotlib import colors, colormaps
from ipywidgets import widgets, Dropdown, RadioButtons, Output, VBox, HBox
from IPython.display import display, clear_output

<h1>Carregando shapefiles (.shp) referentes às arestas, zonas e distritos</h1>

In [2]:
df = pd.read_csv('csvs/2024-01.csv')
dfo = pd.read_csv('csvs-hex/2024-01o.csv')
dfd = pd.read_csv('csvs-hex/2024-01d.csv')

hexs = gpd.read_file('hexs/e6cfcea5a6159226a322a973bf0572da3b847adb803aa53905c4a521abd9a7c4-1718014575357.shp')

edges = gpd.read_file('edges/d71663f99c55ec3269ebde83b71ba106b038c992e6481967d9a95e80e79ba5f6-1716492978093.shp')
edges.crs = 'epsg:4326'

zonas_od = gpd.read_file('OD-2017/Mapas-OD2017/Shape-OD2017/Zonas_2017_region.shp')

zonas_sp = zonas_od[zonas_od['NumeroMuni'] == 36]

# distritos = gpd.read_file('GeoSampa/SIRGAS_SHP_distrito/SIRGAS_SHP_distrito/SIRGAS_SHP_distrito.shp')
# subprefeituras = gpd.read_file('GeoSampa/SIRGAS_SHP_subprefeitura/SIRGAS_SHP_subprefeitura_polygon.shp')
distritos = gpd.read_file('edited shapefiles/edited_distritos.shp')
subprefeituras = gpd.read_file('edited shapefiles/edited_subprefeituras.shp')
distritos.set_crs(epsg=31983, inplace=True)
subprefeituras.set_crs(epsg=31983, inplace=True)

if edges.crs != zonas_sp.crs:
    zonas_sp = zonas_sp.to_crs(edges.crs)
if edges.crs != distritos.crs:
    distritos = distritos.to_crs(edges.crs)
if edges.crs != subprefeituras.crs:
    subprefeituras = subprefeituras.to_crs(edges.crs)

In [3]:
df['total_leisure_trip_count'] = df['forward_leisure_trip_count'] + df['reverse_leisure_trip_count']
df['total_commute_trip_count'] = df['forward_commute_trip_count'] + df['reverse_commute_trip_count']
df['18-34'] = df['forward_18_34_people_count'] + df['reverse_18_34_people_count']
df['35-54'] = df['forward_35_54_people_count'] + df['reverse_35_54_people_count']
df['55-64'] = df['forward_55_64_people_count'] + df['reverse_55_64_people_count']
df['65+'] = df['forward_65_plus_people_count'] + df['reverse_65_plus_people_count']
    
df = df.drop(columns=['forward_leisure_trip_count', 'reverse_leisure_trip_count', 
                                'forward_commute_trip_count', 'reverse_commute_trip_count',
                                'forward_18_34_people_count', 'reverse_18_34_people_count',
                                'forward_35_54_people_count', 'reverse_35_54_people_count',
                                'forward_55_64_people_count', 'reverse_55_64_people_count',
                                'forward_65_plus_people_count', 'reverse_65_plus_people_count'])


<h2>Radio Buttons e Dropdowns</h2>

In [4]:
granularidade = widgets.RadioButtons(
    options=['Geral', 'Zonas', 'Distritos', 'Subprefeituras'],
    value='Geral',
    description='Visualização:',
)

idade = widgets.RadioButtons(
    options=['Qualquer', '18-34', '35-54', '55-64', '65+'],
    value='Qualquer',
    description='Idade:',
)

genero = widgets.RadioButtons(
    options=['Qualquer', 'Feminino', 'Masculino', 'Não especificado'],
    value='Qualquer',
    description='Gênero:',
)

sorted_zonas = zonas_sp.sort_values(by='NomeZona')
zone_dropdown = Dropdown(
    options=[('Selecione uma zona', None)] + 
            [(f"{row['NomeZona']}", row['NumeroZona']) for index, row in sorted_zonas.iterrows()],
    value=None,
    description='Zona:',
)

distritos_dropdown = Dropdown(
    options=[('Selecione um distrito', None)] + 
            [(f"{row['ds_nome']}", row['ds_codigo']) for index, row in distritos.iterrows()],
    value=None,
    description='Distrito:',
)

subprefeituras_dropdown = Dropdown(
    options=[('Selecione uma subprefeitura', None)] + 
            [(f"{row['sp_nome']}", row['sp_codigo']) for index, row in subprefeituras.iterrows()],
    value=None,
    description='Subprefeituras:',
)

state = {'granularidade': granularidade.value, 'idade': idade.value, 'genero': genero.value}

<h2>Zonas OD / Distritos / Subprefeituras</h2>

In [5]:
# Mapa padrão - distritos
m_distritos = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)
folium.GeoJson(
    distritos,
    name='Distritos',
    style_function=lambda feature: {
        'weight': 2,
        'color': 'blue',
        'fillOpacity': 0
    }
).add_to(m_distritos)
folium.LayerControl().add_to(m_distritos)

# Mapa padrão - zonas
m_zonas = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)
folium.GeoJson(
    zonas_sp,
    name='Zonas',
    style_function=lambda feature: {
        'weight': 2,
        'color': 'red',
        'fillOpacity': 0
    }
).add_to(m_zonas)
folium.LayerControl().add_to(m_zonas)

# Mapa padrão - subprefeituras
m_subprefeituras = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)
folium.GeoJson(
    subprefeituras,
    name='Subprefeituras',
    style_function=lambda feature: {
        'weight': 2,
        'color': 'purple',
        'fillOpacity': 0
    }
).add_to(m_subprefeituras)
folium.LayerControl().add_to(m_subprefeituras)

# Outputs
map_output = Output()
table_output = Output()
dropdown_output = Output()
debug_output = Output()

display(HBox([granularidade, idade, genero]), dropdown_output, VBox([map_output, table_output, debug_output]))

def get_edge_color(grouped_edges, edge_uid, min_val, max_val):
    if edge_uid not in grouped_edges['edgeUID'].values:
        norm = colors.Normalize(vmin=min_val, vmax=max_val)
        color_value = norm(min_val)
        return colors.to_hex(cmap_edges(color_value))
    
    total_trips = grouped_edges[grouped_edges['edgeUID'] == edge_uid]['total_trip_count'].values[0]
    norm = colors.Normalize(vmin=min_val, vmax=max_val)
    
    color_value = norm(total_trips)
    return colors.to_hex(cmap_edges(color_value))

# Normalização de valores das escalas
def get_normalization_values(grouped_edges):
    min_val = grouped_edges['total_trip_count'].min()
    max_val = grouped_edges['total_trip_count'].max()
    return min_val, max_val

# Mudança de idade
# def update_age(idade):
#     if idade == "Qualquer":

# Mudança de mapas por granularidade
def update_map(granularidade):
    with map_output:
        map_output.clear_output(wait=True)
        
        if granularidade == "Geral":
            with dropdown_output:
                dropdown_output.clear_output(wait=True)
            display(m_distritos)
            
        elif granularidade == 'Zonas':
            with dropdown_output:
                dropdown_output.clear_output(wait=True)
                display(zone_dropdown)
            display(m_zonas)

        elif granularidade == 'Distritos':
            with dropdown_output:
                dropdown_output.clear_output(wait=True)
                display(distritos_dropdown)
            display(m_distritos)
            
        elif granularidade == 'Subprefeituras':
            with dropdown_output:
                dropdown_output.clear_output(wait=True)
                display(subprefeituras_dropdown)
            display(m_subprefeituras)

def update_loc_wrapper(change):
    age = state['idade']
    update_loc(change, age)

# Mudança de zona/distrito/subprefeitura a partir da seleção do usuário
def update_loc(change, age=None):
    loc_id = change['new']
    if loc_id is None:
        return

    selected_loc_map = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)
    
    if granularidade.value == "Zonas":
        full_map = zonas_sp
        grouped_edges = grouped_edges_in_zones
        sufix = 'da Zona'
        color = 'red'
        selected_loc = zonas_sp[zonas_sp["NumeroZona"] == loc_id]
        name = selected_loc['NomeZona'].iloc[0]
        table_data = trips_per_zone[trips_per_zone['NumeroZona'] == loc_id].reset_index(drop=True)
        table_data = table_data.rename(columns={'NomeZona': 'Nome da Zona', 
                                                'total_trip_count': 'Total de viagens'})
        
    elif granularidade.value == "Distritos":
        full_map = distritos
        grouped_edges = grouped_edges_in_districts
        sufix = 'do Distrito'
        color = 'blue'
        loc_id = str(loc_id)
        selected_loc = distritos[distritos["ds_codigo"] == loc_id]
        name = selected_loc['ds_nome'].iloc[0]
        table_data = trips_per_district[trips_per_district['ds_nome'] == name].reset_index(drop=True)
        table_data = table_data.rename(columns={'ds_nome': 'Nome do distrito', 
                                                'total_trip_count': 'Total de viagens'})
        
    elif granularidade.value == "Subprefeituras":
        full_map = subprefeituras
        grouped_edges = grouped_edges_in_subprefeituras
        sufix = 'da Subprefeitura'
        color = 'purple'
        loc_id = str(loc_id)
        selected_loc = subprefeituras[subprefeituras["sp_codigo"] == loc_id]
        name = selected_loc['sp_nome'].iloc[0]
        table_data = trips_per_subprefeituras[trips_per_subprefeituras['sp_nome'] == name].reset_index(drop=True)
        table_data = table_data.rename(columns={'sp_nome': 'Nome da subprefeitura', 
                                                'total_trip_count': 'Total de viagens'})

    # with debug_output:
    #     debug_output.clear_output(wait=True)  
    #     print(f"reverse commute: {trips_per_subprefeituras['reverse_commute_trip_count']}")
    #     print(f"forward commute: {trips_per_subprefeituras['forward_commute_trip_count']}")
    #     print(f"reverse leisure: {trips_per_subprefeituras['reverse_leisyre_trip_count']}")
    #     print(f"forward leisure: {trips_per_subprefeituras['forward_leisure_trip_count']}")
    #     print(selected_loc)

    # if age is not None and age != "Qualquer":
    #     age = f'{age}'
        
    #     # Filter trips for the selected age group
    #     grouped_edges = grouped_edges[grouped_edges[age] > 0]

    if selected_loc.empty:
        with debug_output:
            print("No matching data found for the selected loc_id.")
        return

    filtered_edges = gpd.overlay(edges, selected_loc, how="intersection")

    selected_edges = grouped_edges[grouped_edges['edgeUID'].isin(filtered_edges['edgeUID'])]
    min_val, max_val = selected_edges['total_trip_count'].min(), selected_edges['total_trip_count'].max()

    # realça a zona/distrito/subprefeitura escolhida
    folium.GeoJson(
        selected_loc,
        style_function=lambda feature: {
            'weight': 2,
            'color': color,
            'fillOpacity': 0
        }
    ).add_to(selected_loc_map)

    # adiciona os demais zona/distrito/subprefeitura
    folium.GeoJson(
        full_map,
        style_function=lambda feature: {
            'weight': 2,
            'color': color,
            'fillOpacity': 0
        }
    ).add_to(selected_loc_map)

    # adiciona os edges correspondente a zona/distrito/subprefeitura escolhida
    folium.GeoJson(
        filtered_edges,
        style_function=lambda feature: {
            'weight': 2,
            'color': get_edge_color(grouped_edges, feature['properties']['edgeUID'], min_val, max_val)
        },
    ).add_to(selected_loc_map)

    # Limites geográficos (bounds) zona/distrito/subprefeitura para focar visualização (zoom)
    bounds = selected_loc.geometry.total_bounds
    sw = [bounds[1], bounds[0]]
    ne = [bounds[3], bounds[2]]
    selected_loc_map.fit_bounds([sw, ne])
    folium.LayerControl().add_to(selected_loc_map)

    with map_output:
        map_output.clear_output(wait=True)
        display(selected_loc_map)

    styled_table = table_data.style \
        .hide(axis='index') \
        .set_table_styles({
            f'Nome {sufix}': [{'selector': 'td, th', 'props': 'width: 200px; text-align: center;'}],
            'Total de viagens': [{'selector': 'td, th', 'props': 'width: 100px; text-align: center;'}]
        }) \
        .set_properties(**{'text-align': 'center', 'padding': '10px'}) \
        .format(thousands=".")

    with table_output:
        table_output.clear_output(wait=True)
        display(styled_table)

def on_value_change(change):
    if change['owner'] is granularidade:
        state['granularidade'] = change['new']
    elif change['owner'] is idade:
        state['idade'] = change['new']
    elif change['owner'] is genero:
        state['genero'] = change['new']
        
    update_map(state['granularidade'])
    update_loc_wrapper(state['idade'])

def compute_trip_counts(grouped_df):
    trips = grouped_df[['total_trip_count', 'total_leisure_trip_count', 'total_commute_trip_count', '18-34', '35-54', '55-64', '65+']].sum().reset_index()
    # trips['total_leisure_trip_count'] = trips['forward_leisure_trip_count'] + trips['reverse_leisure_trip_count']
    # trips['total_commute_trip_count'] = trips['forward_commute_trip_count'] + trips['reverse_commute_trip_count']
    # trips['18-34'] = trips['forward_18_34_people_count'] + trips['reverse_18_34_people_count']
    # trips['35-54'] = trips['forward_35_54_people_count'] + trips['reverse_35_54_people_count']
    # trips['55-64'] = trips['forward_55_64_people_count'] + trips['reverse_55_64_people_count']
    # trips['65+'] = trips['forward_65_plus_people_count'] + trips['reverse_65_plus_people_count']
    
    # trips = trips.drop(columns=['forward_leisure_trip_count', 'reverse_leisure_trip_count', 
    #                             'forward_commute_trip_count', 'reverse_commute_trip_count',
    #                             'forward_18_34_people_count', 'reverse_18_34_people_count',
    #                             'forward_35_54_people_count', 'reverse_35_54_people_count',
    #                             'forward_55_64_people_count', 'reverse_55_64_people_count',
    #                             'forward_65_plus_people_count', 'reverse_65_plus_people_count'])
    
    return trips

# Radio Buttons Observers 
granularidade.observe(on_value_change, names='value')
idade.observe(on_value_change, names='value')
genero.observe(on_value_change, names='value')

# Dropdown Observers
zone_dropdown.observe(update_loc_wrapper, names='value')
distritos_dropdown.observe(update_loc_wrapper, names='value')
subprefeituras_dropdown.observe(update_loc_wrapper, names='value')

update_map(state['granularidade'])

hexs_in_zones = gpd.sjoin(hexs, zonas_sp, how="inner", predicate="intersects")
dfo_merged_zones = pd.merge(dfo, hexs_in_zones[['hex_id', 'NumeroZona', 'NomeZona']], left_on='hex_id', right_on='hex_id', how='inner')
grouped_zones_hex = dfo_merged_zones.groupby(['NumeroZona', 'NomeZona'])

edges_in_zones = gpd.sjoin(edges, zonas_sp, how="inner", predicate="intersects")
df_merged_zones = pd.merge(df, edges_in_zones[['edgeUID', 'NumeroZona', 'NomeZona']], left_on='edge_uid', right_on='edgeUID', how='inner')
grouped_edges_in_zones = df_merged_zones.groupby(['edgeUID', 'NumeroZona', 'NomeZona'])['total_trip_count'].sum().reset_index()
grouped_zones = df_merged_zones.groupby(['NumeroZona', 'NomeZona'])

edges_in_districts = gpd.sjoin(edges, distritos, how="inner", predicate="intersects")
df_merged_districts = pd.merge(df, edges_in_districts[['edgeUID', 'ds_nome', 'ds_codigo']], left_on='edge_uid', right_on='edgeUID', how='inner')
grouped_edges_in_districts = df_merged_districts.groupby(['edgeUID', 'ds_codigo', 'ds_nome'])['total_trip_count'].sum().reset_index()
grouped_districts = df_merged_districts.groupby(['ds_nome'])

edges_in_subprefeituras = gpd.sjoin(edges, subprefeituras, how="inner", predicate="intersects")
df_merged_subprefeituras = pd.merge(df, edges_in_subprefeituras[['edgeUID', 'sp_nome', 'sp_codigo']], left_on='edge_uid', right_on='edgeUID', how='inner')
grouped_edges_in_subprefeituras = df_merged_subprefeituras.groupby(['edgeUID', 'sp_codigo', 'sp_nome'])['total_trip_count'].sum().reset_index()
grouped_subprefeituras = df_merged_subprefeituras.groupby(['sp_nome'])
    
cmap_edges = colormaps.get_cmap('YlOrBr')

trips_hex = grouped_zones_hex['count'].sum().reset_index()
trips_per_zone = compute_trip_counts(grouped_zones)
trips_per_district = compute_trip_counts(grouped_districts)
trips_per_subprefeituras = compute_trip_counts(grouped_subprefeituras)

Output()

In [6]:
print(grouped_edges_in_subprefeituras)

         edgeUID sp_codigo         sp_nome  total_trip_count
0      325297019        20     Parelheiros                15
1      325297020        20     Parelheiros                15
2      325297118        20     Parelheiros                15
3      325297122        20     Parelheiros                15
4      325297123        20     Parelheiros                 5
...          ...       ...             ...               ...
84897  326502790        24  Itaim Paulista                95
84898  326502792        24  Itaim Paulista                95
84899  326502793        24  Itaim Paulista                80
84900  326502794        24  Itaim Paulista                80
84901  326502796        24  Itaim Paulista                95

[84902 rows x 4 columns]


<h2>Quartis</h2>

In [7]:
quartis = widgets.RadioButtons(
    options=['Q1', 'Q2', 'Q3', 'Q4'],
    value='Q1',
    description='Quartil:',
)

quartile_map_output = Output()

display(quartis, quartile_map_output)

trips_per_edge = df.groupby('edge_uid').agg({'total_trip_count': 'sum'}).reset_index()
trips_per_edge['quartile'] = pd.qcut(trips_per_edge['total_trip_count'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
edges_in_quartiles = pd.merge(edges, trips_per_edge[['edge_uid', 'quartile']], left_on='edgeUID', right_on='edge_uid', how='inner')

quartile_colors = {
    'Q1': 'green',
    'Q2': 'blue',
    'Q3': 'orange',
    'Q4': 'red'
}

def update_quartile(change):
    quartiles_map = folium.Map(location=[-23.5789, -46.6388], tiles="CartoDB Positron", zoom_start=11)
    selected_quartile = change['new']
    
    filtered_edges = edges_in_quartiles[edges_in_quartiles['quartile'] == selected_quartile]
    
    for _, row in filtered_edges.iterrows():
        folium.GeoJson(
            row['geometry'],
            style_function=lambda feature, color=quartile_colors[selected_quartile]: {
                'color': color,
                'weight': 2,
                'opacity': 1,
            }
        ).add_to(quartiles_map)

    with quartile_map_output:
        quartile_map_output.clear_output(wait=True)
        display(quartiles_map)
    
quartis.observe(update_quartile, names='value')

update_quartile({'new': quartis.value})

RadioButtons(description='Quartil:', options=('Q1', 'Q2', 'Q3', 'Q4'), value='Q1')

Output()